### File to compile results at both intraoperative and preoperative level for all outcomes

#### Input: pred_stage, outcome name, result file location 

In [2]:
import pickle
import json
import os
import numpy as np
import pandas as pd
import math
from datetime import datetime
import matplotlib.pyplot as plt
import pprint

In [13]:
# result location 
pred_stage = 'Preoperative'  # {'Preoperative', 'Intraoperative'}
outcome_list= ['icu', 'mortality']
sav_dir = '/home/trips/PeriOperative_RiskPrediction/Best_results/' + pred_stage +"/"

In [14]:
for outcome in outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + '.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
    model_name_list = list(metric_data.keys())
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_auroc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_auprc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)

    for key, value in metric_data.items():
        for ab in ablation_name_list:
            df_auroc.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
            df_auprc.loc[ab, key] = np.round(np.mean(value[ab][:,1]), decimals=3)

    # renaming the index
    df_auroc = df_auroc.rename(index=dict(zip(df_auroc.index, ablation_name_list_pretty)))
    df_auprc = df_auprc.rename(index=dict(zip(df_auprc.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_auroc.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_auroc.columns = new_column_index
    df_auprc.columns = new_column_index

    print(" Outcome ", outcome, " AUROC")
    pprint.pp(df_auroc)
    print("\n Outcome ", outcome, " AUPRC")
    pprint.pp(df_auprc)

 Outcome  icu  AUROC
                                     icu              
                                  TabNet  Scarf   XGBT
preops_cbow                        0.898  0.909  0.969
preops_cbow_homemeds               0.921  0.824  0.974
preops_cbow_pmh_problist_homemeds  0.912  0.877  0.974

 Outcome  icu  AUPRC
                                     icu              
                                  TabNet  Scarf   XGBT
preops_cbow                        0.642  0.649  0.885
preops_cbow_homemeds               0.693  0.369  0.896
preops_cbow_pmh_problist_homemeds  0.651  0.502  0.895
 Outcome  mortality  AUROC
                                  mortality              
                                       XGBT  Scarf TabNet
preops_cbow                           0.943  0.897    0.9
preops_cbow_homemeds                  0.947   0.89  0.912
preops_cbow_pmh_problist_homemeds     0.945  0.873  0.919

 Outcome  mortality  AUPRC
                                  mortality              
    

In [15]:
# result location 
pred_stage = 'Intraoperative'  # {'Preoperative', 'Intraoperative'}
outcome_list= ['icu', 'mortality']
sav_dir = '/home/trips/PeriOperative_RiskPrediction/Best_results/' + pred_stage +"/"

In [18]:
for outcome in outcome_list:
    perf_filename = sav_dir + str(outcome) + '_Best_perf_metrics_combined_' + pred_stage.lower() + '.pickle'
    with open(perf_filename, 'rb') as file:
        metric_data = pickle.load(file)
    model_name_list = list(metric_data.keys())
    ablation_name_list = list(metric_data[model_name_list[0]].keys())
    ablation_name_list_pretty = [i.split("_",1)[1] for i in ablation_name_list]
    
    df_auroc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)
    df_auprc = pd.DataFrame(columns=model_name_list, index=ablation_name_list)

    for key, value in metric_data.items():
        for ab in ablation_name_list:
            try:
                df_auroc.loc[ab, key] = np.round(np.mean(value[ab][:,0]), decimals=3)
                df_auprc.loc[ab, key] = np.round(np.mean(value[ab][:,1]), decimals=3)
            except(KeyError):
                df_auroc.loc[ab, key] = 'Yet to be filled'
                df_auprc.loc[ab, key] = 'Yet to be filled'

    # renaming the index
    df_auroc = df_auroc.rename(index=dict(zip(df_auroc.index, ablation_name_list_pretty)))
    df_auprc = df_auprc.rename(index=dict(zip(df_auprc.index, ablation_name_list_pretty)))
    
    new_columns = [(outcome, col) for col in df_auroc.columns]
    new_column_index = pd.MultiIndex.from_tuples(new_columns)
    
    df_auroc.columns = new_column_index
    df_auprc.columns = new_column_index

    print(" Outcome ", outcome, " AUROC")
    pprint.pp(df_auroc)
    print("\n Outcome ", outcome, " AUPRC")
    pprint.pp(df_auprc)

 Outcome  icu  AUROC
                                                      icu                  
                                                     MVCL       transformer
meds                                                0.846  Yet to be filled
flow                                                0.924  Yet to be filled
flow_meds                                           0.931  Yet to be filled
preops_flow_meds                                    0.973  Yet to be filled
preops_homemeds_flow_meds                           0.974  Yet to be filled
preops_pmh_problist_homemeds_flow_meds              0.975             0.927
preops_pmh_problist_homemeds_flow_meds_postopco...  0.932  Yet to be filled

 Outcome  icu  AUPRC
                                                      icu                  
                                                     MVCL       transformer
meds                                                0.488  Yet to be filled
flow                                         